# 0. Install and Import Dependencies

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [2]:
# Optional if you are using a GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# 1. Load Model

In [3]:
model = hub.load('https://tfhub.dev/google/movenet/multipose/lightning/1')
movenet = model.signatures['serving_default']

# 2. Functions

In [4]:
# Function to loop through each person detected and render
def loop_through_people(frame, keypoints_with_scores, edges, confidence_threshold):
    for person in keypoints_with_scores:
        draw_connections(frame, person, edges, confidence_threshold)
        draw_keypoints(frame, person, confidence_threshold)

In [5]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 6, (0,255,0), -1)

In [6]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 4)

In [8]:
cap = cv2.VideoCapture(1)

while cap.isOpened():
    ret, frame = cap.read()
    
    # Recolor Feed
    #image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    #image.flags.writeable = False
    
    # Recolor image back to BGR for rendering
    #image.flags.writeable = True
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    #img = image.copy()
    #img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    #input_img = tf.cast(img, dtype=tf.int32)
    
    # Resize image
    img = frame.copy()
    img = tf.image.resize_with_pad(tf.expand_dims(img, axis=0), 384,640)
    input_img = tf.cast(img, dtype=tf.int32)
    
    # Detection section
    results = movenet(input_img)
    keypoints_with_scores = results['output_0'].numpy()[:,:,:51].reshape((6,17,3))
    
    # Render keypoints
    loop_through_people(frame, keypoints_with_scores, EDGES, 0.1)
    cv2.imshow('Raw Webcam Feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [9]:
keypoints_with_scores

array([[[7.53860235e-01, 5.28118312e-01, 5.53468704e-01],
        [6.77463293e-01, 5.76493442e-01, 5.44870615e-01],
        [6.70402408e-01, 4.92911339e-01, 5.39301813e-01],
        [6.88417792e-01, 6.42193496e-01, 4.12806839e-01],
        [6.83336139e-01, 4.51187372e-01, 4.89011765e-01],
        [9.06734109e-01, 7.17524350e-01, 1.68323055e-01],
        [9.45770681e-01, 3.70509237e-01, 8.87161195e-02],
        [9.71021533e-01, 7.19109356e-01, 1.31790491e-03],
        [9.77956235e-01, 3.35384756e-01, 1.88992750e-02],
        [9.76266503e-01, 6.45220995e-01, 3.85162421e-02],
        [8.72806251e-01, 4.51710910e-01, 1.79003645e-02],
        [9.94419992e-01, 6.16849899e-01, 6.41118269e-03],
        [9.85371470e-01, 4.79049444e-01, 1.67957265e-02],
        [9.18169975e-01, 6.17218196e-01, 1.17570453e-03],
        [8.42655361e-01, 4.66771603e-01, 1.34431003e-02],
        [4.02331114e-01, 5.39959610e-01, 4.51973453e-03],
        [4.79891986e-01, 4.61263746e-01, 1.06454850e-03]],

       [[7.5

In [10]:
keypoints_with_scores = keypoints_with_scores[:4]
practice = np.delete(keypoints_with_scores, 2, 2)

In [11]:
practice

array([[[0.75386024, 0.5281183 ],
        [0.6774633 , 0.57649344],
        [0.6704024 , 0.49291134],
        [0.6884178 , 0.6421935 ],
        [0.68333614, 0.45118737],
        [0.9067341 , 0.71752435],
        [0.9457707 , 0.37050924],
        [0.97102153, 0.71910936],
        [0.97795624, 0.33538476],
        [0.9762665 , 0.645221  ],
        [0.87280625, 0.4517109 ],
        [0.99442   , 0.6168499 ],
        [0.9853715 , 0.47904944],
        [0.91817   , 0.6172182 ],
        [0.84265536, 0.4667716 ],
        [0.4023311 , 0.5399596 ],
        [0.479892  , 0.46126375]],

       [[0.75386024, 0.5281183 ],
        [0.6774633 , 0.57649344],
        [0.6704024 , 0.49291134],
        [0.6907604 , 0.6410427 ],
        [0.68333614, 0.45118737],
        [0.92413014, 0.6762765 ],
        [0.93303394, 0.38846478],
        [0.93756616, 0.66960776],
        [0.9462638 , 0.34033996],
        [0.9364711 , 0.66140246],
        [0.92981064, 0.4261868 ],
        [0.9343536 , 0.60251445],
        [0.9

In [12]:
finalPractice = np.delete(practice, [0,1,2,3,4], 1)

In [13]:
finalPractice

array([[[0.9067341 , 0.71752435],
        [0.9457707 , 0.37050924],
        [0.97102153, 0.71910936],
        [0.97795624, 0.33538476],
        [0.9762665 , 0.645221  ],
        [0.87280625, 0.4517109 ],
        [0.99442   , 0.6168499 ],
        [0.9853715 , 0.47904944],
        [0.91817   , 0.6172182 ],
        [0.84265536, 0.4667716 ],
        [0.4023311 , 0.5399596 ],
        [0.479892  , 0.46126375]],

       [[0.92413014, 0.6762765 ],
        [0.93303394, 0.38846478],
        [0.93756616, 0.66960776],
        [0.9462638 , 0.34033996],
        [0.9364711 , 0.66140246],
        [0.92981064, 0.4261868 ],
        [0.9343536 , 0.60251445],
        [0.93066514, 0.49066004],
        [0.9277026 , 0.5623246 ],
        [0.92149705, 0.40900746],
        [0.7594389 , 0.46753463],
        [0.7574646 , 0.47953627]],

       [[0.89990866, 0.7212359 ],
        [0.8894867 , 0.44159403],
        [0.94944763, 0.7441544 ],
        [0.949892  , 0.41118965],
        [0.95459557, 0.68722934],
        [0

In [14]:
finalPractice[0]

array([[0.9067341 , 0.71752435],
       [0.9457707 , 0.37050924],
       [0.97102153, 0.71910936],
       [0.97795624, 0.33538476],
       [0.9762665 , 0.645221  ],
       [0.87280625, 0.4517109 ],
       [0.99442   , 0.6168499 ],
       [0.9853715 , 0.47904944],
       [0.91817   , 0.6172182 ],
       [0.84265536, 0.4667716 ],
       [0.4023311 , 0.5399596 ],
       [0.479892  , 0.46126375]], dtype=float32)

In [15]:
vectorList = [
    [0,1],
    [0,2],
    [1,3],
    [2,4],
    [3,5],
    [0,6],
    [1,7],
    [6,7],
    [6,8],
    [7,9],
    [8,10],
    [9,10]
]

In [16]:
for j in vectorList:
    print(j[0])
    print(j[1])
    print("====")

0
1
====
0
2
====
1
3
====
2
4
====
3
5
====
0
6
====
1
7
====
6
7
====
6
8
====
7
9
====
8
10
====
9
10
====


In [17]:
finalCoordinate = []
for person in finalPractice:
    tempPerson = []
    for i in vectorList:
#         print(person[i[0]])    # first coordinate (y,x)
#         print(person[i[1]])    # second coordinate (y,x)
        tempVector = person[i[1]] - person[i[0]]
        tempPerson.append(tempVector)
        print(person[i[1]] - person[i[0]])    # vector(second - first)
        print("====")
    finalCoordinate.append(tempPerson)

[ 0.03903657 -0.3470151 ]
====
[0.06428742 0.00158501]
====
[ 0.03218555 -0.03512448]
====
[ 0.00524497 -0.07388836]
====
[-0.10514998  0.11632615]
====
[ 0.08768588 -0.10067445]
====
[0.03960079 0.10854021]
====
[-0.00904852 -0.13780046]
====
[-0.07625002  0.0003683 ]
====
[-0.14271611 -0.01227784]
====
[-0.51583886 -0.07725859]
====
[-0.44032425  0.07318801]
====
[ 0.0089038  -0.28781173]
====
[ 0.01343602 -0.00666875]
====
[ 0.01322985 -0.04812482]
====
[-0.00109506 -0.00820529]
====
[-0.01645315  0.08584684]
====
[ 0.01022345 -0.07376206]
====
[-0.00236881  0.10219526]
====
[-0.00368845 -0.1118544 ]
====
[-0.00665098 -0.04018986]
====
[-0.00916809 -0.08165258]
====
[-0.16826373 -0.09478995]
====
[-0.16205817  0.05852717]
====
[-0.01042199 -0.27964184]
====
[0.04953897 0.02291852]
====
[ 0.06040531 -0.03040439]
====
[ 0.00514793 -0.05692506]
====
[-0.00171202  0.07791716]
====
[ 0.05565572 -0.06001312]
====
[0.06081235 0.06612757]
====
[-0.00526536 -0.15350115]
====
[-0.01088011 -0.

In [18]:
np.squeeze(finalCoordinate)

array([[[ 3.90365720e-02, -3.47015113e-01],
        [ 6.42874241e-02,  1.58500671e-03],
        [ 3.21855545e-02, -3.51244807e-02],
        [ 5.24497032e-03, -7.38883615e-02],
        [-1.05149984e-01,  1.16326153e-01],
        [ 8.76858830e-02, -1.00674450e-01],
        [ 3.96007895e-02,  1.08540207e-01],
        [-9.04852152e-03, -1.37800455e-01],
        [-7.62500167e-02,  3.68297100e-04],
        [-1.42716110e-01, -1.22778416e-02],
        [-5.15838861e-01, -7.72585869e-02],
        [-4.40324247e-01,  7.31880069e-02]],

       [[ 8.90380144e-03, -2.87811726e-01],
        [ 1.34360194e-02, -6.66874647e-03],
        [ 1.32298470e-02, -4.81248200e-02],
        [-1.09505653e-03, -8.20529461e-03],
        [-1.64531469e-02,  8.58468413e-02],
        [ 1.02234483e-02, -7.37620592e-02],
        [-2.36880779e-03,  1.02195263e-01],
        [-3.68845463e-03, -1.11854404e-01],
        [-6.65098429e-03, -4.01898623e-02],
        [-9.16808844e-03, -8.16525817e-02],
        [-1.68263733e-01, -9.4

In [19]:
finalCoordinate[0]

[array([ 0.08807701, -0.31244847], dtype=float32),
 array([ 0.10449451, -0.01100016], dtype=float32),
 array([ 0.02446681, -0.02244526], dtype=float32),
 array([-0.06557697, -0.04400718], dtype=float32),
 array([-0.11848861,  0.09877566], dtype=float32),
 array([ 0.12705684, -0.06787813], dtype=float32),
 array([0.01834333, 0.10865113], dtype=float32),
 array([-0.0206365 , -0.13591921], dtype=float32),
 array([-0.15769094,  0.0353632 ], dtype=float32),
 array([-0.1645711 ,  0.03651595], dtype=float32),
 array([-0.27322966, -0.09388238], dtype=float32),
 array([-0.245713  ,  0.04088408], dtype=float32)]

In [21]:
vectors_only_body = np.array(finalCoordinate)

In [22]:
vectors_only_body

array([[[ 3.90365720e-02, -3.47015113e-01],
        [ 6.42874241e-02,  1.58500671e-03],
        [ 3.21855545e-02, -3.51244807e-02],
        [ 5.24497032e-03, -7.38883615e-02],
        [-1.05149984e-01,  1.16326153e-01],
        [ 8.76858830e-02, -1.00674450e-01],
        [ 3.96007895e-02,  1.08540207e-01],
        [-9.04852152e-03, -1.37800455e-01],
        [-7.62500167e-02,  3.68297100e-04],
        [-1.42716110e-01, -1.22778416e-02],
        [-5.15838861e-01, -7.72585869e-02],
        [-4.40324247e-01,  7.31880069e-02]],

       [[ 8.90380144e-03, -2.87811726e-01],
        [ 1.34360194e-02, -6.66874647e-03],
        [ 1.32298470e-02, -4.81248200e-02],
        [-1.09505653e-03, -8.20529461e-03],
        [-1.64531469e-02,  8.58468413e-02],
        [ 1.02234483e-02, -7.37620592e-02],
        [-2.36880779e-03,  1.02195263e-01],
        [-3.68845463e-03, -1.11854404e-01],
        [-6.65098429e-03, -4.01898623e-02],
        [-9.16808844e-03, -8.16525817e-02],
        [-1.68263733e-01, -9.4

In [28]:
for person in vectors_only_body:
    print(person)
    print("====================")
    for vector in person:
        print(vector)
        print("====")

[[ 3.90365720e-02 -3.47015113e-01]
 [ 6.42874241e-02  1.58500671e-03]
 [ 3.21855545e-02 -3.51244807e-02]
 [ 5.24497032e-03 -7.38883615e-02]
 [-1.05149984e-01  1.16326153e-01]
 [ 8.76858830e-02 -1.00674450e-01]
 [ 3.96007895e-02  1.08540207e-01]
 [-9.04852152e-03 -1.37800455e-01]
 [-7.62500167e-02  3.68297100e-04]
 [-1.42716110e-01 -1.22778416e-02]
 [-5.15838861e-01 -7.72585869e-02]
 [-4.40324247e-01  7.31880069e-02]]
[ 0.03903657 -0.3470151 ]
====
[0.06428742 0.00158501]
====
[ 0.03218555 -0.03512448]
====
[ 0.00524497 -0.07388836]
====
[-0.10514998  0.11632615]
====
[ 0.08768588 -0.10067445]
====
[0.03960079 0.10854021]
====
[-0.00904852 -0.13780046]
====
[-0.07625002  0.0003683 ]
====
[-0.14271611 -0.01227784]
====
[-0.51583886 -0.07725859]
====
[-0.44032425  0.07318801]
====
[[ 0.0089038  -0.28781173]
 [ 0.01343602 -0.00666875]
 [ 0.01322985 -0.04812482]
 [-0.00109506 -0.00820529]
 [-0.01645315  0.08584684]
 [ 0.01022345 -0.07376206]
 [-0.00236881  0.10219526]
 [-0.00368845 -0.11185

In [64]:
import math

lamda = 1

BPD = []
# d = []

for i in range(12):
    tempBodyPart = []
    tempD = []
    
    # Calculate vector
    for person in vectors_only_body:
        tempBodyPart.append(person[i])
        print(person[i])
        print("====")
    tempBodyPart = np.array(tempBodyPart)
    tempAverageVector = tempBodyPart.mean(axis = 0)
    
    # Calculate d
    for vi in tempBodyPart:
        tempDVal = np.linalg.norm(vi - tempAverageVector)
        tempD = np.array(tempD)
        tempD = np.append(tempD, tempDVal)
    
    
    BPD = np.array(BPD)
    BPD = np.append(BPD, math.pow(tempD.mean(), lamda))
        
#     d.append(tempD.mean())
#     print(d)
print(BPD)
print("===============")

[ 0.03903657 -0.3470151 ]
====
[ 0.0089038  -0.28781173]
====
[-0.01042199 -0.27964184]
====
[-0.01825919  0.00180636]
====
[0.06428742 0.00158501]
====
[ 0.01343602 -0.00666875]
====
[0.04953897 0.02291852]
====
[-0.0096698   0.00765624]
====
[ 0.03218555 -0.03512448]
====
[ 0.01322985 -0.04812482]
====
[ 0.06040531 -0.03040439]
====
[0.00435652 0.0018263 ]
====
[ 0.00524497 -0.07388836]
====
[-0.00109506 -0.00820529]
====
[ 0.00514793 -0.05692506]
====
[-0.00920952 -0.00459053]
====
[-0.10514998  0.11632615]
====
[-0.01645315  0.08584684]
====
[-0.00171202  0.07791716]
====
[-0.00718406 -0.00119406]
====
[ 0.08768588 -0.10067445]
====
[ 0.01022345 -0.07376206]
====
[ 0.05565572 -0.06001312]
====
[-0.00731725 -0.00587645]
====
[0.03960079 0.10854021]
====
[-0.00236881  0.10219526]
====
[0.06081235 0.06612757]
====
[ 0.00607345 -0.00726109]
====
[-0.00904852 -0.13780046]
====
[-0.00368845 -0.1118544 ]
====
[-0.00526536 -0.15350115]
====
[-0.00486849  0.00042173]
====
[-0.07625002  0.00

In [38]:
a = np.array([[1,2],[3,4]])
a.mean(axis=0)

array([2., 3.])

In [39]:
b = np.array([1,1])
np.linalg.norm(b)

1.4142135623730951

In [41]:
c = np.array([1,2,3])
c.mean()

2.0